### Get VK api Post

In [28]:
import requests
from datetime import datetime, timedelta
import time
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [30]:
ACCESS_TOKEN = '5816254b5816254b5816254b7558614893558165816254b386fa62025fb312bc944f6cc'
APP_ID = 7826904
API_URL = 'https://api.vk.com/method'
V = '5.91'

In [42]:
def get_comment(owner_id, comment_id):
    api_method = f'{API_URL}/wall.getComment'
    resp = requests.get(api_method, params={
        'access_token': ACCESS_TOKEN,
        'owner_id': f'{owner_id}',
        'comment_id': f'{comment_id}',
        'v': V
    })
    data = resp.json()
#     data = resp.json()['response'][0]
    return data
post = get_comment(90688576, 2183)
post

{'error': {'error_code': 28,
  'error_msg': 'Application authorization failed: method is unavailable with service token.',
  'request_params': [{'key': 'owner_id', 'value': '90688576'},
   {'key': 'comment_id', 'value': '2183'},
   {'key': 'v', 'value': '5.91'},
   {'key': 'method', 'value': 'wall.getComment'},
   {'key': 'oauth', 'value': '1'}]}}

In [39]:
def get_post(wall_id, post_id):
    api_method = f'{API_URL}/wall.getById'
    resp = requests.get(api_method, params={
        'access_token': ACCESS_TOKEN,
        'posts': f'{wall_id}_{post_id}',
        'v': V
    })
    data = resp.json()['response'][0]
    return data
post = get_post(96331273, 51555)
post

{'id': 51555,
 'from_id': 96331273,
 'owner_id': 96331273,
 'date': 1660203727,
 'post_type': 'post',
 'text': 'Главное к этому часу:\n\n▪️Около 2,5 тысяч жителей Бердянска в Запорожской области получили паспорта РФ, еще 5 тысяч человек ожидают получения гражданства, сообщил начальник миграционной службы УВД Бердянска Василий Бараненко\n\n▪️В Пхеньяне называли причиной вспышки COVID-19 «странный предмет» на границе с Южной Кореей. Сестра Ким Чен Ына Ким Ё Чжон пообещала уничтожить власти Южной Кореи, если они будут «провоцировать» проникновение вируса в ее страну.\n\n▪️Зеленский: Чем мощнее будет наше оружие и чем больше будет дальность его применения, тем быстрее закончится эта жестокая война\n\n▪️Одно судно с зерном не смогло выйти из украинского Черноморска из-за погодных условий, сообщили в Минобороны Турции...',
 'attachments': [{'type': 'photo',
   'photo': {'album_id': -7,
    'date': 1660203712,
    'id': 457294589,
    'owner_id': 96331273,
    'access_key': '8af6387638aa5708f

### Check mysql_connection

In [14]:
import mysql.connector

client = mysql.connector.connect(
  host="localhost",
  user="danil",
  password="24092022"
)

#             host=constants.db_host,
#             user=constants.db_user,
#             password=constants.db_password

print(client)

### Создать базу данных

In [15]:
mycursor = client.cursor()
mycursor.execute("CREATE DATABASE IF NOT EXISTS diplom")
mycursor.execute("USE diplom")

### Создать таблицу постов

In [6]:
# mycursor.execute("drop table Posts")
mycursor.execute("""
CREATE TABLE IF NOT EXISTS Posts (
    ID VARCHAR(20) PRIMARY KEY,
    date DATETIME,
    text VARCHAR(4000),
    count_comments INT,
    count_likes INT,
    count_reporsts INT,
    count_views INT
)""")
mycursor.execute("desc Posts")
mycursor.fetchall()

[('ID', b'varchar(20)', 'NO', 'PRI', None, ''),
 ('date', b'datetime', 'YES', '', None, ''),
 ('text', b'varchar(4000)', 'YES', '', None, ''),
 ('count_comments', b'int', 'YES', '', None, ''),
 ('count_likes', b'int', 'YES', '', None, ''),
 ('count_reporsts', b'int', 'YES', '', None, ''),
 ('count_views', b'int', 'YES', '', None, '')]

### Вставить пост в таблицу

In [7]:
insert = "INSERT INTO Posts VALUES (%s, FROM_UNIXTIME(%s), %s, %s, %s, %s, %s)"
values = [(90688576_2227, 1462812355, "Brutus", 1, 1, 1, 1)]
    
mycursor.executemany(insert, values)

client.commit()

In [8]:
def insert_post(post):
    insert = "INSERT IGNORE INTO Posts VALUES (%s, FROM_UNIXTIME(%s), %s, %s, %s, %s, %s)"
    
    values = [f"{post['owner_id']}_{post['id']}", post['date'], post['text'], 
               post['comments']['count'], post['likes']['count'], 
               post['reposts']['count'], 
               post['views']['count'] if 'views' in post else None]
    
    mycursor.execute(insert, values)

insert_post(post)
#f"{post['owner_id']}_{post['id']}"

In [9]:
(f"{post['owner_id']}_{post['id']}", post['date'], post['text'], post['comments']['count'], post['likes']['count'], post['reposts']['count'], post['views']['count'] if 'views' in post else None)

('90688576_2227', 1462812355, '', 0, 2, 0, None)

In [12]:
mycursor.close()

False

### Тоже самое для комментов. 1) Найти все не прокомментированные 